In [29]:
!pip install matplotlib_inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')
import scipy.stats as ss
from scipy import optimize
from scipy.optimize import minimize, LinearConstraint, NonlinearConstraint
import pandas as pd
from scipy.integrate import quad

**Task 1**

In [31]:
data = pd.read_csv('ocdrug.csv', index_col='Subject')

G = {}
for i in [1, 2]:
    for j in [1, 2]:
        G[(i, j)] = np.log(data[(data['Sequence'] == i) & (data['Period'] == j)])

def K(x, y):
  n = len(x)
  m = len(y)
  s_x = x.std(ddof=1)
  s_y = y.std(ddof=1)
  return ((s_x**2 / n) + (s_y**2 / m)) ** 2 / ((s_x**2 / n)**2/(n - 1) + (s_y**2 / m)**2/(m - 1))


alpha = 0.05
for a in ['EE', 'NET']:
    for b in ['AUC', 'CMax']:
        c = f'{a} {b}'
        x, y = (G[1, 1][c] - G[1, 2][c]) / 2, (G[2, 1][c] - G[2, 2][c]) / 2
        k = K(x, y) 
        interval = np.array([ss.t(k).ppf(1-alpha), ss.t(k).ppf(alpha)])
        q = x.std(ddof=1)**2/len(x) +  y.std(ddof=1)**2/len(y)
        interval = (x.mean() - y.mean()) - np.sqrt(q) * interval
        print(interval)

[0.10104017 0.20323299]
[0.01168579 0.17100061]
[-0.0265861   0.03883735]
[-0.24940176 -0.04575348]


In [32]:
[np.log(0.8), np.log(1.25)]

[-0.2231435513142097, 0.22314355131420976]

**Task 2**

In [33]:
data = pd.read_csv('ocdrug.csv', index_col='Subject')

G = {}
for i in [1, 2]:
    for j in [1, 2]:
        G[(i, j)] = np.log(data[(data['Sequence'] == i) & (data['Period'] == j)])


z = pd.concat([G[1, 1]["NET CMax"] - G[1, 2]["NET CMax"], G[2, 2]["NET CMax"] - G[2, 1]["NET CMax"]])

def make_ml(z):
  def f(theta):
    loc, scale = theta
    return -ss.t(3,loc=loc, scale=scale).logpdf(z).sum()
  return f


res = minimize(make_ml(z), x0=np.array([0,1]), bounds=[(None, None), (0.0001, None)])
loc_, scale_ = res.x
n = 22

def g(x):
  res = minimize(make_ml(x), x0=np.array([0,1]), bounds=[(None, None), (0.0001, None)])
  loc, scale = res.x
  return np.sqrt(n) * (loc / scale)

alpha = 0.05
N = 1000
samples = ss.t(3).rvs(size=(N, n), random_state=100)
sample = np.array([g(x) for x in samples])
x_L, x_R = np.quantile(sample, alpha), np.quantile(sample, 1 - alpha)
loc_ - (scale_ * x_R) / np.sqrt(n), loc_ - (scale_ * x_L) / np.sqrt(n)

(-0.16846289654641022, -0.033158177337341596)

**Task 3**

In [37]:
freqs_obs = np.array([13, 14, 18, 18, 15, 7, 6, 9])
n = freqs_obs.sum()
bins = [[0, 1, 2], [3], [4], [5], [6], [7], [8]]
def freqs_exp(mu):
        p = np.zeros(len(bins) + 1)
        for i, b in enumerate(bins):
            p[i] = ss.poisson.pmf(b, mu=mu).sum()
        p[-1] = 1 - p.sum()
        return p

lambda_star = 5.028603917186539

ss.chisquare(freqs_obs, n * freqs_exp(5), ddof=0).pvalue, ss.chisquare(freqs_obs, n * freqs_exp(lambda_star), ddof=1).pvalue

(0.9631075721498039, 0.9267223232615678)

**Task 4**

In [38]:
with open('tokyo_farmers.txt', 'r') as file:
    data = []
    for line in file:
        data.append(list(map(int, line.split())))

data = np.array(data).ravel()
vals, counts = np.unique(data, return_counts=True)
# vals = [0, 1, 2, 3, 4, 5, 7, 9]
# counts = [584, 398, 168,  35,   9,   4,   1,   1]
bins1 = [[0], [1], [2], [3], [4]]
bins2 = [[0], [1], [2], [3]]
n = counts.sum()
freqs1 = np.array([584, 398, 168,  35,   9,  6])
freqs2 = np.array([584, 398, 168,  35,   15])

def fun_maker(freqs, bins):
    def chi2(mu):
        p = np.zeros(len(bins) + 1)
        for i, b in enumerate(bins):
            p[i] = ss.poisson.pmf(b, mu=mu).sum()
        p[-1] = 1 - p.sum()
        return np.sum((freqs - p) ** 2 / p)
    return chi2

def freqs_exp(mu, bins):
        p = np.zeros(len(bins) + 1)
        for i, b in enumerate(bins):
            p[i] = ss.poisson.pmf(b, mu=mu).sum()
        p[-1] = 1 - p.sum()
        return p

x0 = 1
bounds = [(0.1, 10)]
res1 = minimize(fun_maker(freqs1, bins1), x0, bounds=bounds)
res2 = minimize(fun_maker(freqs2, bins2), x0, bounds=bounds)
lambda1 = res1.x[0]
lambda2 = res2.x[0]

ss.chisquare(freqs1, n * freqs_exp(lambda1, bins1), ddof=1).pvalue, ss.chisquare(freqs2, n * freqs_exp(lambda2, bins2), ddof=1).pvalue

(0.0006894678644963628, 0.053471669650999185)

**Task 5**

In [39]:
n = 13
m = 37
N = 100000
samples = ss.norm().rvs(size=(N,n + m), random_state=42)
def stat(z):
  return ss.mannwhitneyu(z[:n], z[n:]).statistic
stats = np.array([stat(sample) for sample in samples])
np.quantile(stats, 0.95)

315.0